<a href="https://colab.research.google.com/github/mingd00/Face-Recognition/blob/main/EfficientNet(with_FER).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EfficientNet 모델을 활용한 FER 데이터 학습



### 0. 라이브러리 호출

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import cv2

### 1. 모델 로드

- 모델 기본 구조 확인

In [4]:
# 모델 로드 (ImageNet 가중치 사용)
model = EfficientNetB0(weights='imagenet', include_top=True)

# 모델 구조 출력
model.summary()

21834768/21834768 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "efficientnetb0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 224, 224, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 224, 224, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 32)   │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 32)   │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 32)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 5,330,571 (20.33 MB)

 Trainable params: 5,288,548 (20.17 MB)

 Non-trainable params: 42,023 (164.16 KB)

- 모델 수정: 사전 학습된 가중치는 고정하고 새로운 출력 레이어만 학습

In [17]:
# 1. EfficientNetB0 로드 (사전 학습된 ImageNet 가중치 사용)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# 2. 모델 수정 (FER 데이터셋에 맞게 출력 레이어 조정)
x = base_model.output
x = GlobalAveragePooling2D()(x)  # 공간 차원을 평균화하여 벡터로 변환
x = Dense(128, activation='relu')(x)  # 추가 Fully Connected Layer
predictions = Dense(7, activation='softmax')(x)  # FER 데이터셋의 7개 감정 클래스 출력

model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 48, 48, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_2 (Rescaling)   │ (None, 48, 48, 3)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_1           │ (None, 48, 48, 3)      │              7 │ rescaling_2[0][0]      │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_3 (Rescaling)   │ (None, 48, 48, 3)      │              0 │ normalization_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 49, 49, 3)      │              0 │ rescaling_3[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 24, 24, 32)     │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 24, 24, 32)     │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 24, 24, 32)     │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 24, 24, 32)     │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 24, 24, 32)     │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 24, 24, 32)     │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 4,214,442 (16.08 MB)

 Trainable params: 4,172,419 (15.92 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [18]:
# 3. 사전 학습된 가중치 잠금 (Feature Extraction)
for layer in base_model.layers:
    layer.trainable = False  # EfficientNet 가중치 고정

# 4. 컴파일
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### 2. FER 데이터 로드 및 준비

In [21]:
# FER 데이터셋 로드
data = pd.read_csv("fer2013.csv")

# 데이터 및 라벨 분리
pixels = data['pixels'].str.split(" ").tolist()  # 픽셀 데이터를 리스트로 변환
pixels = np.array(pixels, dtype='float32') / 255.0  # 정규화
pixels = pixels.reshape(-1, 48, 48, 1)  # 48x48 그레이스케일 이미지 형태로 변환
pixels = np.repeat(pixels, 3, axis=-1)  # 그레이스케일 이미지를 3채널로 확장 (EfficientNet 입력 요구사항)

labels = to_categorical(data['emotion'], num_classes=7)

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(pixels, labels, test_size=0.2, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((28709, 48, 48, 3), (7178, 48, 48, 3), (28709, 7), (7178, 7))

### 3. 학습

In [22]:
# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

train_generator = datagen.flow(X_train, y_train, batch_size=32)
val_generator = ImageDataGenerator().flow(X_val, y_val, batch_size=32)

In [23]:
# 첫 번째 학습: Feature Extraction (사전 학습된 가중치 고정)
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(X_train) // 32,
    validation_steps=len(X_val) // 32
)

Epoch 1/10


  self._warn_if_super_not_called()



897/897 ━━━━━━━━━━━━━━━━━━━━ 79s 62ms/step - accuracy: 0.2387 - loss: 1.8271 - val_accuracy: 0.2457 - val_loss: 1.8131
Epoch 2/10
  1/897 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.1875 - loss: 1.9121

  self.gen.throw(typ, value, traceback)



897/897 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.1875 - loss: 1.9121 - val_accuracy: 0.4000 - val_loss: 1.6642
Epoch 3/10
897/897 ━━━━━━━━━━━━━━━━━━━━ 48s 42ms/step - accuracy: 0.2465 - loss: 1.8157 - val_accuracy: 0.2458 - val_loss: 1.8191
Epoch 4/10
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 28us/step - accuracy: 0.2500 - loss: 1.8182 - val_accuracy: 0.3000 - val_loss: 1.7206
Epoch 5/10
897/897 ━━━━━━━━━━━━━━━━━━━━ 38s 42ms/step - accuracy: 0.2573 - loss: 1.8095 - val_accuracy: 0.2460 - val_loss: 1.8102
Epoch 6/10
897/897 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1875 - loss: 1.9155 - val_accuracy: 0.2000 - val_loss: 1.8307
Epoch 7/10
897/897 ━━━━━━━━━━━━━━━━━━━━ 39s 41ms/step - accuracy: 0.2463 - loss: 1.8181 - val_accuracy: 0.2457 - val_loss: 1.8122
Epoch 8/10
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 44us/step - accuracy: 0.2812 - loss: 1.7933 - val_accuracy: 0.4000 - val_loss: 1.6728
Epoch 9/10
897/897 ━━━━━━━━━━━━━━━━━━━━ 40s 44ms/step - accuracy: 0.2556 - loss: 1.8119 - val_accuracy: 0.2

In [ ]:
# 두 번째 학습: Fine-Tuning (가중치 조정 가능하게 변경)
for layer in base_model.layers:
    layer.trainable = True  # EfficientNet 가중치 조정 가능하게 변경

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=10,
                    steps_per_epoch=len(X_train) // 32,
                    validation_steps=len(X_val) // 32
                    )

Epoch 1/10
619/897 ━━━━━━━━━━━━━━━━━━━━ 13s 50ms/step - accuracy: 0.5524 - loss: 1.1726

### 4. 모델 평가

In [ ]:
# 손실 함수
plt.figure(figsize=(5, 3))
plt.plot(history.history['loss'],'r',linewidth=2.0)
plt.plot(history.history['val_loss'],'b',linewidth=2.0)
plt.legend(['Training loss', 'Validation Loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Curves')

# 정확도
plt.figure(figsize=(5, 3))
plt.plot(history.history['accuracy'],'r',linewidth=2.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.xlabel('Epochs ')
plt.ylabel('Accuracy')
plt.title('Accuracy Curves')

plt.show()

### 5. 예측

In [11]:
img_path = 'img.jpg'  # 이미지 파일 경로
img = image.load_img(img_path, target_size=(224, 224))  # EfficientNetB0의 입력 크기로 맞춤
img_array = image.img_to_array(img)  # 이미지를 numpy 배열로 변환
img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가
img_array = preprocess_input(img_array)  # EfficientNet에 맞는 전처리 적용
img_array

array([[[[255., 223., 224.],
         [255., 223., 224.],
         [255., 223., 224.],
         ...,
         [255., 223., 224.],
         [255., 223., 224.],
         [254., 224., 224.]],

        [[255., 223., 224.],
         [255., 223., 224.],
         [255., 223., 224.],
         ...,
         [255., 223., 224.],
         [255., 223., 224.],
         [254., 224., 224.]],

        [[255., 223., 224.],
         [255., 223., 224.],
         [255., 223., 224.],
         ...,
         [255., 223., 224.],
         [255., 223., 224.],
         [254., 224., 224.]],

        ...,

        [[252., 250., 253.],
         [252., 250., 253.],
         [252., 250., 253.],
         ...,
         [245., 241., 242.],
         [245., 241., 242.],
         [241., 240., 238.]],

        [[251., 249., 252.],
         [251., 249., 252.],
         [251., 249., 252.],
         ...,
         [245., 241., 242.],
         [244., 240., 241.],
         [240., 239., 237.]],

        [[250., 248., 251.],
       

### 3. 예측

In [13]:
pred = model.predict(img_array)
decoded_pred = decode_predictions(pred, top=5)
decoded_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


[[('n04584207', 'wig', 0.922464),
  ('n03595614', 'jersey', 0.00875497),
  ('n04599235', 'wool', 0.0050379355),
  ('n03476991', 'hair_spray', 0.003515503),
  ('n02786058', 'Band_Aid', 0.0025500865)]]

In [15]:
for i, (imagenet_id, label, score) in enumerate(decoded_pred[0]):
    print(f"{i + 1}: {label} ({score:.2f})")

1: wig (0.92)
2: jersey (0.01)
3: wool (0.01)
4: hair_spray (0.00)
5: Band_Aid (0.00)
